In [ ]:
%matplotlib inline
import pynbody
import pynbody.plot.sph as sph
import pynbody.plot as pp
from pynbody.analysis import profile
import matplotlib.pylab as plt
import numpy as np
import os
import bokeh
from bokeh.layouts import gridplot, row, column
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

bokeh.io.output_notebook()

In [ ]:
from astropy import cosmology
import astropy.units as u
cosmo = cosmology.Planck15
print(cosmo)
RHO_C = cosmo.critical_density0.to('solMass/kpc**3')
RHO_C

In [ ]:
@u.quantity_input
def halo_Wechsler2002(M: u.solMass):
    '''Return the NFW concentration factor following Wechsler et al 2002 fitting formula
    M is in solar mass'''
    c = 20 * (M/(1e11 * u.solMass))**-0.13
    return c

@u.quantity_input
def halo_Strigari2007(M: u.solMass):
    '''Return the NFW concentration factor following Strigari et al 2007 fitting formula
    M is in solar mass.

    This formula is more suited for dwarf Galaxies (M < 1e8 Msol)'''
    c = 33 * (M/(1e8 * u.solMass))**-0.06
    return c

def halo_scaled_density(c, rho_c=RHO_C, overdensity_factor=200.0):
    rho_s = overdensity_factor * c*c*c * rho_c / (3 * np.log(1+c) - c/(1+c))
    return rho_s  # Msol/km^3

def halo_scaled_radius(M, c, rho_c=RHO_C, overdensity_factor=200.0):
    R_s = ((M / (4/3 * np.pi * overdensity_factor * rho_c)) ** (1. / 3)) / c
    return R_s  # km


In [ ]:
M_h = 1e14 * u.solMass
c = halo_Wechsler2002(M_h)
c

In [ ]:
virial_radius = c * halo_scaled_radius(M_h, c)
virial_radius

In [ ]:
import tqdm
def compute_cog(snapshots, directory='', save_cache=True, cache_file="cog.npz"):
    import collections
    if not isinstance(snapshots, collections.Iterable):
        snapshots = [snapshots]
#     print("Processing {} files".format(len(snapshots)))

    cog = np.zeros((3, len(snapshots)), dtype=float)
    times = np.zeros(len(snapshots), dtype=float)
    i = 0
    for snap in tqdm.tqdm(snapshots):
        # duck-typing in case the snapshots are strings
        if not isinstance(snap, pynbody.gadget.GadgetSnap):
            sim = pynbody.load(os.path.join(directory, snap))
        else:
            sim = snap
#         print("snapshot time {} Gyr".format(sim.properties['time'].in_units('Gyr')))
        times[i] = sim.properties['time'].in_units('Gyr')
        mass = sim['mass']
        pos = sim['pos']
        
        tot_mass = mass.sum()
        # print "tot mass = {} Msol".format(tot_mass)
        cog[:,i] = np.array([(pos[:,0] * mass).sum(), (pos[:,1] * mass).sum(), (pos[:,2] * mass).sum()])/tot_mass

        i += 1
#     print(cog)
    if save_cache:
        np.savez(cache_file, times=times, cog=cog)
    return times, cog


def snapshot_list(dirname, stem="snapshot_", fillwidth=4, include_dir=False):
    import os
    import glob
    if not os.path.isdir(dirname):
         raise IOError("{} is not a directory".format(dirname))
    if include_dir:
         filelist = glob.glob(os.path.join(dirname, stem) + "*")
    else:
         filelist = list(map(os.path.basename, glob.glob(os.path.join(dirname, stem) + "*")))
    filelist.sort()
    return filelist

In [ ]:
SIM_NUMBER = 71002

In [ ]:
# sim_path = '/home/michele/sim/MoRIA/sim62002/'
# sim_path = '/home/michele/sim/nfw_negative/out'


moria_path = '/home/michele/sim/MoRIA/'
kicked_path = '/home/michele/sim/MySimulations/Moria8Gyr_tidal'

def load_sim(path_to_snapshots):
    snaplist = snapshot_list(path_to_snapshots, include_dir=True)
    simlist = [pynbody.load(snap) for snap in snaplist]
    return simlist

def load_moria_sim_and_kicked(sim_number, kicked=True, moria_path=moria_path, kicked_path=kicked_path):
    snaps_path = os.path.join(moria_path, 'sim{}'.format(sim_number))
    simlist = load_sim(snaps_path)
    
    if kicked:
        ksnaps_path = os.path.join(kicked_path, 'sim{}'.format(sim_number), 'out')
        ksimlist = load_sim(ksnaps_path)
        return simlist, ksimlist
    else:
        return simlist

In [ ]:
moria_sim, kicked_sim = load_moria_sim_and_kicked(SIM_NUMBER)

In [ ]:
# sim_path = '/home/michele/sim/MySimulations/Moria8Gyr_tidal/sim60002/out/'
# snaplist = snapshot_list(sim_path, include_dir=True)
# sim = pynbody.load(snaplist[-1])
# sim.physical_units()
# print(sim.filename)
# print(sim.properties['time'].in_units('Gyr'))
# sim.properties
# Jtot = np.sqrt(((np.multiply(sim['j'].transpose(), sim['mass']).sum(axis=1))**2).sum()) # calculate angular momentum

In [ ]:
# p_all = profile.Profile(s62, max='250 kpc')
# plt.plot(p_all['rbins'].in_units('kpc'),p_all['vr_disp'].in_units('km s^-1'),'k')
# plt.xlabel('$R$ [kpc]'); plt.ylabel('$\sigma_{r}$')

In [ ]:
recompute = False
# npy_file = 'cog.npy'
if not recompute:
    data = np.load('cog{}.npz'.format(SIM_NUMBER))
    times_moria, cog_moria = data['times'], data['cog']
    data = np.load('cog{}_kicked.npz'.format(SIM_NUMBER))
    times_kicked, cog_kicked = data['times'], data['cog']
else:
    times_moria, cog_moria = compute_cog(moria_sim, cache_file='cog{}.npz'.format(SIM_NUMBER));
    times_kicked, cog_kicked = compute_cog(kicked_sim, cache_file='cog{}_kicked.npz'.format(SIM_NUMBER));

Use trange to equalize the range of the simulation between the two 

In [ ]:
bins_sfr = 500
t_range = (np.min(times_moria))
# sfr, t = pynbody.plot.sfh(sim, bins=bins_sfr)
# f, ax1 = plt.subplots(1, figsize=(10,6))

In [ ]:
def sfh(simlist, bins=100, **kwargs):
    # Take the last snapshot
    sim = simlist[-1]
    sfr, t = pynbody.plot.sfh(sim, bins=bins_sfr, **kwargs)
    return sfr, t

In [ ]:
sfr, sfr_bins = sfh(moria_sim, bins=bins_sfr) # , subplot=ax1)
ksfr, ksfr_bins = sfh(kicked_sim, bins=bins_sfr) # , subplot=ax1)
sfr_bins
# np.testing.assert_array_equal(sfr_bins, ksfr_bins)
sfr_bins[-1], ksfr_bins[-1]

# plt.plot(t[:-1], sfr)

In [ ]:
if recompute:
    del moria_sim, kicked_sim
    import gc
    gc.collect()

In [ ]:
cog_moria.shape

In [ ]:
times_moria.shape

In [ ]:
def rebin_cog(sfr_times, sim_times, cog):
    rebin_x_cog = np.interp(sfr_times, sim_times, cog[0,:])
    rebin_y_cog = np.interp(sfr_times, sim_times, cog[1,:])
    return rebin_x_cog, rebin_y_cog

In [ ]:
sfr_t_moria = sfr_bins[:-1]
sfr_t_kicked = ksfr_bins[:-1]

x_cog_moria, y_cog_moria = rebin_cog(sfr_t_moria, times_moria, cog_moria)
x_cog_kicked, y_cog_kicked = rebin_cog(sfr_t_kicked, times_kicked, cog_kicked)

In [ ]:
x_cog, y_cog = cog_moria[0,:], cog_moria[1,:]

In [ ]:
source_moria = ColumnDataSource(data=dict(t=sfr_t_moria,
                                          sfr=sfr,
                                          x_cog=x_cog_moria,
                                          y_cog=y_cog_moria))
source_moria.column_names

In [ ]:
source_kicked = ColumnDataSource(data=dict(t=sfr_t_kicked,
                                           sfr=ksfr,
                                           x_cog=x_cog_kicked,
                                           y_cog=y_cog_kicked))
# t = source_kicked.data['t']


In [ ]:
from bokeh.models.annotations import Span
from bokeh.models import HoverTool

def compare_cog_sfh(source):
    start = (source.data['x_cog'][0], source.data['y_cog'][0])
    end = (source.data['x_cog'][-1], source.data['y_cog'][-1])

    hover = HoverTool(tooltips=[
#             ("(x,y)", "($x_cog, $y_cog)"),
            ("time", "@t"),
        ],
                      names=["cog"])

    tools = 'lasso_select, box_select, tap, wheel_zoom, pan, box_zoom, reset'
    
    # COG figure
    f1 = figure(width=400, height=400, title='Center of gravity position',
                x_axis_label="x (kpc)", y_axis_label="y (kpc)",
                tools=tools)
    f1.circle('x_cog', 'y_cog', source=source, name='cog')
    f1.circle(*start, color='yellow')
    f1.circle(*end, color='red')
    f1.circle(0,0, radius=virial_radius.value, alpha=0.1)
    f1.x(0,0)
    f1.add_tools(hover) # do not hover on the virial radius!

    # SFH figure
    f2 = figure(width=400, height=400, title='SFH',
                x_axis_label="Time (Gyr)", y_axis_label="SFR (Msol/yr)",
                tools=tools)
    # f2.tools.pop(4)  # remove pan
    
    # f2.line(t[:-1], sfr)
    f2.vbar(x='t', top='sfr', width=.1, source=source)
    
    
#     t = source.data['t']
#     sfr = source.data['sfr']
#     interval = t[1]-t[0]
#     f2.quad(top='sfr', bottom=np.zeros_like(t), left=t,
#             right=t+interval, alpha=0.5)#, color="#B3DE69")
    simulation_start_line = Span(location=times_moria[0], dimension='height', line_color='firebrick', line_width=1)
    f2.add_layout(simulation_start_line)
    p = row([f1, f2])
    # p = gridplot([[f1, f2]])
    show(p)
    return f1, f2

In [ ]:
compare_cog_sfh(source_moria)

In [ ]:
f1, f2 = compare_cog_sfh(source_kicked)

In [ ]:
print(f2.tools)
f2.tools.pop(4)

In [ ]:
# f1 = figure(width=400, height=400, title='Center of gravity position', x_axis_label="x (kpc)", y_axis_label="y (kpc)")
# f1.line(x_cog, y_cog)
# f1.circle(0,0, radius=virial_radius.value, alpha=0.1)
# f2 = figure(width=400, height=400, x_axis_label="Time (Gyr)", y_axis_label="SFR (Msol/yr)")
# # f2.line(t[:-1], sfr)
# f2.vbar(x=t[:-1], top=sfr, width=None)
# p = row([f1, f2])
# show(p)

In [ ]:
# from bokeh.layouts import layout, row, widgetbox
# from bokeh.models import (
#     ColumnDataSource, HoverTool, TextInput, SingleIntervalTicker, Slider, Button, Label,
#     CategoricalColorMapper,
# )

# def animate_update():
#     gyr = slider.value + 1
#     if gyr > years[-1]:
#         gyr = years[0]
#     slider.value = gyr

# def animate():
#     if button.label == '► Play':
#         button.label = '❚❚ Pause'
#         curdoc().add_periodic_callback(animate_update, 200)
#     else:
#         button.label = '► Play'
#         curdoc().remove_periodic_callback(animate_update)

# button = Button(label='► Play', width=60)
# button.on_click(animate)

In [ ]:
import numpy as np
from bokeh.layouts import layout, row, widgetbox
from bokeh.models import (
    ColumnDataSource, HoverTool, TextInput, SingleIntervalTicker, Slider, Button, Label,
    CategoricalColorMapper,
)
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.plotting import figure, show

def modify_doc(doc):

    # Set up data
    N = 200
    x = np.linspace(0, 4*np.pi, N)
    y = np.sin(x)
    source = ColumnDataSource(data=dict(x=x, y=y))


    # Set up plot
    plot = figure(plot_height=400, plot_width=400, title="my sine wave",
                  tools="crosshair,pan,reset,save,wheel_zoom",
                  x_range=[0, 4*np.pi], y_range=[-2.5, 2.5])

    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    # Set up widgets
    text = TextInput(title="title", value='my sine wave')
    offset = Slider(title="offset", value=0.0, start=-5.0, end=5.0, step=0.1)
    amplitude = Slider(title="amplitude", value=1.0, start=-5.0, end=5.0, step=0.1)
    phase = Slider(title="phase", value=0.0, start=0.0, end=2*np.pi)
    freq = Slider(title="frequency", value=1.0, start=0.1, end=5.1, step=0.1)

    # Set up callbacks
    def update_title(attrname, old, new):
        plot.title.text = text.value

    text.on_change('value', update_title)

    def update_data(attrname, old, new):

        # Get the current slider values
        a = amplitude.value
        b = offset.value
        w = phase.value
        k = freq.value

        # Generate the new curve
        x = np.linspace(0, 4*np.pi, N)
        y = a*np.sin(k*x + w) + b

        source.data = dict(x=x, y=y)

    for w in [offset, amplitude, phase, freq]:
        w.on_change('value', update_data)

    # Set up layouts and add to document
    inputs = widgetbox(text, offset, amplitude, phase, freq)

    doc.add_root(row(inputs, plot, width=800))
    doc.title = "Sliders"

    # Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)


In [ ]:
doc = app.create_document()
# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")